In [ ]:
import numpy as np
from sklearn.experimental import enable_hist_gradient_boosting

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords 
import re
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.linear_model import SGDClassifier, RidgeClassifierCV
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.semi_supervised import SelfTrainingClassifier

from sklearn.semi_supervised import LabelPropagation
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import NearestCentroid

from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.decomposition import PCA

from sklearn.feature_extraction.text import CountVectorizer
import pickle
from gensim.models.doc2vec import TaggedDocument, Doc2Vec
from gensim.sklearn_api import D2VTransformer

from sklearn.feature_extraction.text import HashingVectorizer
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.decomposition import LatentDirichletAllocation

import nltk
import string

In [ ]:
def clean_name(name):
    name = re.sub(r'[^\w]', ' ', name)
    name = re.sub(r'[,:._\-\[\]\d]', ' ', name)
    
    s = ''
    for i in tokenize(name.lower().strip()):
        s += ' ' + i
    return s#name.lower().strip()

def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 3:
                continue
            tokens.append(word.lower())
    return tokens

def tokenize(text):
    stem = nltk.stem.SnowballStemmer('english')
    text = text.lower()
    
    for token in nltk.word_tokenize(text):
        if token in string.punctuation:
            continue
        yield stem.stem(token)

def find_outlets(clf, X_train):
    pred = clf.predict(X_train)
    d = train.copy()
    d['pred'] = pred
    
    return d[d.category_id != d.pred].groupby('category_id').apply(len).sort_values(), d

def write_submit(freq, clf):
    pickle.dump(freq, open('submit_names/tfidf', 'wb'))
    pickle.dump(clf, open('submit_names/clf_task1', 'wb'))
    !cd submit_names;zip submit_names.zip *  
    
def read_and_clean_data():
    train = pd.read_parquet('data_fusion_train.parquet')
    train = train[train.category_id != -1].drop_duplicates('item_name')
    train['item_name'] = train['item_name'].apply(clean_name)

    train.replace('', np.nan, inplace=True)
    train.dropna(subset = ['item_name'], inplace = True)

    return train.sample(frac=1)   

def train_model(train):
    X_train = freq.fit_transform(train.item_name)

    clf = LinearSVC()
    print(cross_val_score(clf, X_train, train.category_id, cv=3, scoring='f1_weighted'))
    _ = clf.fit(X_train, train.category_id)
    return clf, X_train

def category_id_hyperport(data):
    iter = 1
    cats = data.category_id.unique()
    results = []

    for c1 in cats:
        for c2 in cats[iter:]:
            df = data.copy()
            df = df[(df.category_id != c1)&(df.category_id != c2)]
            train, test = train_test_split(df, test_size=0.3, stratify=df['category_id'])
            freq = CountVectorizer(stop_words=stop, binary = True)
            clf, X_train = train_model(train)
            pred = clf.predict(freq.transform(test.item_name))
            score = f1_score(test.category_id, pred, average='weighted')
            print('iter {}, score {}, excluded categories {},{}, train size {}'.format(iter, score, c1, c2, len(train)))
            results.append({'score' : score, 'cat1' : c1, 'cat2' : c2})
        iter += 1

In [ ]:
train = read_and_clean_data()

drop_outlets = True
stop = stopwords.words('russian') 
freq = CountVectorizer(stop_words=stop, binary = True)

In [ ]:
clf, X_train = train_model(train)

In [ ]:
if drop_outlets == True:
    outlets, train_no_outlets = find_outlets(clf, X_train)
    train_no_outlets = train_no_outlets[train_no_outlets.category_id == train_no_outlets.pred]
    train_no_outlets.sample(frac=1) 
    clf, X_train = train_model(train_no_outlets)
    
write_submit(freq, clf)

# This is draft

In [ ]:
d[(d.category_id == 61)&(d.category_id != d.pred)]

In [ ]:
d[(d.category_id == 133)&(d.item_name.str.contains('белье'))]

In [ ]:
d[(d.category_id == 61)&(d.item_name.str.contains('белье'))]

In [ ]:
array([0.59400355, 0.60206136, 0.59848624]) - NearestCentroid
array([0.7039181 , 0.70752764, 0.71109001]) - RandomForestClassifier
array([0.71958694, 0.72747844, 0.7260711 ]) - ExtraTreesClassifier
array([0.73099794, 0.73521632, 0.73163451]) - MultinomialNB(alpha = 0.05)
array([0.73408719, 0.74478913, 0.74937629]) - SGDClassifier
array([0.75503983, 0.75194788, 0.7556436 ]) - MLPClassifier
array([0.753369  , 0.75643443, 0.7608544 ]) - PassiveAggressiveClassifier
array([0.75609124, 0.75369773, 0.76303259]) - RidgeClassifierCV
array([0.77044554, 0.77085181, 0.77058879]) - BaggingClassifier(LinearSVC())
array([0.77754205, 0.77519272, 0.77945042]) - LinearSVC
array([0.77099336, 0.77438669, 0.78064815]) - CalibratedClassifierCV
array([0.80577326, 0.80291802, 0.80744682]) - CalibratedClassifierCV, freq

In [ ]:
df = pd.DataFrame.from_dict([freq.vocabulary_]).T
df.columns = ['goods']
df['size'] = df.index.str.len()
d = df.sort_values('goods')
d